In [ ]:
import sys

from tqdm import tqdm
import json
import torch

raw_data_path = '/workspace/data/corpus/raw_corpus_bpe.txt'
raw_data_path = '/workspace/data/corpus/first_5_lines_bpe.txt'
output_dir = '/workspace/exp'/

vocab_path = '/workspace/data/vocabs/chord.json'
with open(vocab_path, 'r') as file:
    vocab = json.load(file)

raw_data = []
with open(raw_data_path, 'r') as f:
    for line in tqdm(f, desc="reading original txt file..."):
        raw_data.append(line.strip())


In [5]:
import time, os, json
from collections import Counter
from pprint import pprint
from tqdm import tqdm
import subprocess
from functools import partial
from p_tqdm import p_uimap
RATIO = 4
MERGE_CNT = 700
CHAR_CNT = 128
WORKERS = 32

def count_rhythm_token(toks, ratio=RATIO):
    # 1개의 곡에 대해서 수행하는 함수
    if isinstance(toks, str):
        toks = toks.split()

    mulpies = dict()
    chord_dict = Counter()
    l_toks = len(toks)

    chord_list = []
    
    for idx in range(0, l_toks, ratio):
        t1, t2, t3, t4 = toks[idx : idx + 4]
        
        if t1[0] == 'h' or t1[0] == 'H':
            chord_list.append(t1)
        
    chord_start = ''
    chord_mid = ''
    chord_end = ''    
    for idx, chord in enumerate(chord_list):
        # Init first case
        if idx == 0:
            chord_start = chord
            continue
        
        if idx == 1:
            chord_mid = chord
            continue
        
        chord_end = chord
        
        if chord_start + chord_mid + chord_end == '':
            continue
        
        chord_dict[chord_start+chord_mid+chord_end] += 1
        chord_start = chord_mid
        chord_mid = chord_end
        

    return chord_dict, l_toks // ratio


if __name__ == '__main__':
    start_time = time.time()

    paragraphs = []

    raw_data_path = '/workspace/data/corpus/raw_corpus_bpe.txt'
    # raw_data_path = '/workspace/data/corpus/first_5_lines_bpe.txt'
    # output_dir = '../../data/corpus/'
    output_dir = '/workspace/exp/'
    os.makedirs(output_dir, exist_ok=True)
    raw_data = []
    with open(raw_data_path, 'r') as f:
        for line in tqdm(f, desc="reading original txt file..."):
            raw_data.append(line.strip())

    chord_dict = Counter()
    before_total_tokens = 0
    for sub_chord_dict, l_toks in p_uimap(count_rhythm_token, raw_data, num_cpus=WORKERS):
        chord_dict += sub_chord_dict
        before_total_tokens += l_toks
    
    mulpi_list = sorted(chord_dict.most_common(), key=lambda x: (-x[1], x[0]))
    with open(output_dir+'chord_group.txt', 'w') as f:
        f.write(str(len(mulpi_list)) + '\n')
        for k, v in mulpi_list:
            f.write(''.join(k) + ' ' + str(v) + '\n')


reading original txt file...: 46188it [00:15, 2934.14it/s]
100%|██████████| 46188/46188 [02:11<00:00, 351.17it/s]
